<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/Teste_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rotinas de Criação de Matriz de Embeddings


In [74]:
! pip install keras==2.7.0
! pip show keras

     |████████████████████████████████| 1.3 MB 27.4 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
tensorflow 2.8.0 requires keras<2.9,>=2.8.0rc0, but you have keras 2.7.0 which is incompatible.


Name: keras
Version: 2.7.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: tensorflow, keras-vis


In [75]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [91]:
import numpy
import itertools
import nltk
import string, re
import pandas as pd
from numpy import array
from numpy import asarray
from numpy import loadtxt
from numpy import savetxt
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text, sequence

In [92]:
########## PATHS DE ARQUIVOS ##########
# 
PATH_CBOW_s300 = r"/content/drive/MyDrive/TCC/dados/word_embeddings/cbow_s300_modifyed.txt"

#
PATH_BASE_1_CLASSIFICADA = r"/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1JuntaHateENotHateDaLeila_balanceada.csv"
PATH_MATRIZ_CBOW_300_BASE_1 = r"/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base1JuntaHateENotHateDaLeila_balanceada_CBOW300.CSV"
#
PATH_BASE_2_CLASSIFICADA = r"/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_maior4_menor25_limpissima_balanceada.csv"
PATH_MATRIZ_CBOW_300_BASE_2 = r"/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base2_maior4_menor25_limpissima_balanceada_CBOW300.CSV"

In [93]:
#DataFrame_Base= pd.read_csv(PATH_BASE_1_CLASSIFICADA)
DataFrame_Base = pd.read_csv(PATH_BASE_2_CLASSIFICADA)
DataFrame_Base.head()

,Unnamed: 0,label,text
0,0,1,ambiguinho eu estou com nojo
1,1,1,menino do céu para que está feio
2,2,1,já pergunto a porra do trem já sabendo da vdd ...
3,3,1,sardella só faz merda
4,4,1,foda se na real vou tratar igual


In [105]:
palavras_unicas = list(DataFrame_Base['text'].str.split(' ', expand=True).stack().unique())
print(len(palavras_unicas))
# retira eventuais numeros. Numeros unicos por algum motivo matam o processo de filtragem. 
palavras_unicas = set(map(lambda x: re.findall(r'^\D+', x)[0] if re.findall(r'^\D+', x) != [] else "a" ,palavras_unicas))
print(len(palavras_unicas))

9144
9081


# Filtra os embeddings

In [95]:
# Código para filtrar o arquivo de embeddings.

embeddings_index = dict()

f = open(PATH_CBOW_s300, encoding='utf-8')

for line in f:
    values = line.split()
    word = values[0]
    if (word in palavras_unicas):
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
f.close()

In [96]:
def monta_linha(palavra, coeficientes):
    # Monta a linha com palavra _ coeficientes.
    
    linha = f'{palavra}'
    for coef in coeficientes:
        linha += f' {coef:.5f}'
    linha += '\n'
    
    return linha

In [97]:
lista_de_embeddings = []
for word in embeddings_index:
    coeficientes = embeddings_index[word]
    linha = monta_linha(word, coeficientes)
    lista_de_embeddings.append(linha)

In [98]:
embeddings_dict = {}
for i, line in enumerate(lista_de_embeddings):
    values = line.split()
    embeddings_dict[values[0]] = numpy.asarray(values[1:], dtype='float32')

In [99]:
token = text.Tokenizer()
token.fit_on_texts(palavras_unicas)
#word_index: dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.
word_index = token.word_index

In [100]:
# create token-embedding mapping

lista_de_chaves = list(embeddings_dict.keys())

tamanho_da_dimensao = len(embeddings_dict[lista_de_chaves[0]])

embedding_matrix = numpy.zeros((len(word_index) + 1, tamanho_da_dimensao ))

for word, i in word_index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [101]:
print(type(embedding_matrix))

<class 'numpy.ndarray'>


In [102]:
print(embedding_matrix)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.17693999 -0.04666    -0.148      ... -0.27713001  0.17879
   0.0385    ]
 [ 0.40718001 -0.28492001  0.00271    ... -0.27092999 -0.21365
   0.07886   ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.44192001 -0.45881999 -0.18363    ... -0.09414    -0.29789999
  -0.40656   ]]


In [103]:
# save to csv file
savetxt(PATH_MATRIZ_CBOW_300_BASE_2, embedding_matrix, delimiter=',')

In [104]:
# data = loadtxt('data.csv', delimiter=',')
# # print the array
# print(data)